# Learning 1d Pure Advection in Conservative Form

To verify that our PINN implementation is working well, we test the PINN on the following problem:
$$
    \begin{cases}
        u_t + au_x = 0\\
        \lim_{|x|\rightarrow\infty}u(t, x) = 0\\
        u(0, x) = u_0(x)
    \end{cases}
$$ where $u_0 = \mathcal{N}(x|0, 1)$. 

In this case, the NN solution is assumed to have the following model (conservative advection):
$$
    \frac{\partial u_{\theta_1}}{\partial t} + \frac{\partial}{\partial x}\bigg(\mathcal{V}_{\theta_2}(x)u_{\theta_1}\bigg) = 0
$$

The data is generated in `constant_advection.ipynb`.

In [ ]:
from PINN.AdvectionNet import *
# Testing
import matplotlib.pyplot as plt
import torch
import numpy as np
import scipy

# set random seeds
np.random.seed(10)
torch.manual_seed(10);

import numpy as np    
import matplotlib.pyplot as plt

# load data
data_path = "../data/LinearOscillator/Pure_Advection_small_data.mat"
# create PINN
pinn = AdvectionNet(indim=2, outdim=1, data_path=data_path, scheduler="ExponentialLR")
# testing training
info = train(
    pinn, pinn.optimizer, pinn.scheduler, batch_size=2**10,
    epochs=200, batch_print=100, mode="all",
    conservative_pde=True
)

In [ ]:
# visualize losses
plt.figure(1, figsize=(16, 8));
plt.plot(np.log10(info["pde_loss"]), lw=2, color="red");
plt.title("PDE Loss vs. Epoch");

plt.figure(2, figsize=(16, 8));
plt.plot(np.log10(info["data_loss"]), lw=2, color="blue");
plt.title("Data Loss vs. Epoch");

query = cartesian_data(pinn.tgrid, pinn.xgrid)
p_pred = pinn(query).reshape(pinn.nx, pinn.nt).T.detach().numpy()

# grids
tgrid = pinn.tgrid.detach().numpy()
xgrid = pinn.xgrid.detach().numpy()

p_exact = pinn.raw_data["pmc"]
dx = pinn.dx.item()
dt = pinn.dt.item()

In [ ]:
nx = 400
xgrid_fine = np.linspace(-10.0, 10.0, nx)
xgrid_fine = torch.tensor(xgrid_fine)
dx = xgrid_fine[1]-xgrid_fine[0]
nt = 1000
tgrid_fine = np.linspace(0., 10.0, nt)
tgrid_fine = torch.tensor(tgrid_fine)
dt = tgrid_fine[1]-tgrid_fine[0]

query_fine = cartesian_data(tgrid_fine, xgrid_fine)

G_pred_fine = pinn.G_nn(query_fine[:, 1][:, None]).reshape(nx, nt).T.detach().numpy()
p_pred_fine = pinn(query_fine).reshape(nx, nt).T.detach().numpy()

In [ ]:
plt.contourf(G_pred_fine); plt.colorbar();

In [ ]:
plt.figure(1);
plt.plot(xgrid_fine, G_pred_fine[0, :], color="red");
plt.plot(xgrid_fine, np.ones_like(xgrid_fine), "--", color="blue", lw=5.0, alpha=0.5, label="exact");
plt.ylim([0.9, 1.1])
plt.legend(); 
plt.xlabel(r"$x$");
plt.ylabel(r"$G(x)$");